## libraries and function 

In [ ]:
# !pip install impyute
!pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time 
# from missingpy import MissForest

# Test bivariate normality

In [ ]:
!pip install scipy 

  Using cached scipy-1.7.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install pingouin
from pingouin import multivariate_normality

def non_normal_rate(X, alpha = 0.05):
  # permute through all pairs of features:
  d = X.shape[1]
  n_non_normal = 0
  for i in range(d):
    for j in range(i):
      res = multivariate_normality(X[:,[i,j]], alpha=.05)
      if res.pval < alpha:
        n_non_normal += 1
  return n_non_normal/d/(d-1)/2

### MLE estimation function 

In [ ]:
def diag_term(X,i):
  arr0 = X[:,i].flatten()
  arr = arr0[~np.isnan(arr0)]
  return np.var(arr)

def musMLE(X,y,G):
    n,p = X.shape[0], X.shape[1]
    f = lambda g: np.nanmean(X[y==g,:],axis=0)
    musMLE = np.array([f(g) for g in range(G)])    
    return musMLE.T

def Smle(X,y,musMLE,g):
    '''
    function to compute the covariance matrix for the g-th class
    X: input, should be a numpy array
    y: label
    G: number of classes
    g: class index
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    Xg, yg = X[y==g,:], y[y==g]
    n,p = Xg.shape[0], Xg.shape[1] 
 
    S = np.diag([diag_term(Xg,i) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        if ((S[i,i] == 0.) | (S[j,j] == 0.)):
          S[i,j] = S[j,i] = 0.
          continue

        mat = Xg[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], yg[idx]
        A = mg = len(y_arr) 

        s11 = mg*np.var(mat[:,0])
        s22 = mg*np.var(mat[:,1])
        s12 = sum((mat[:,0]-musMLE[i,g])*(mat[:,1]-musMLE[j,g]))
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)
        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0
 
        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1: 
          if sum(r==0.0) == len(r):
            r = 0.
          else:  
            w = np.cov(mat, rowvar=False)  
            #r = r[w[0,1]*r>=0]
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 

        S[i,j] = S[j,i] = r
    return S

### compute_err function 

In [ ]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape) 

In [ ]:
def compute_err_mle(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  run_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])
    # MLEs approach
    start = time.time()
    mus_mle = musMLE(Xtr_nan,y,G)
    S_mle = np.array([Smle(Xtr_nan,y,mus_mle, g) for g in range(G)])   
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start  
    e_rate.append(mle_err)
    run_time.append(mle_time)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate), np.mean(np.asarray(run_time))

In [ ]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  run_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    print('mus soft impute', mus_softimpute)
    print('S soft impute', S_softimpute)
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start
    e_rate.append(softimpute_err)
    run_time.append(softimpute_time)

  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate),np.mean(np.asarray(run_time))

### MNIST

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Fetch the dataset directly
mnist = tfds.image.MNIST()
# or by string name
mnist = tfds.builder('mnist')

# Download the data, prepare it, and write it to disk
mnist.download_and_prepare()

# Load data from disk as tf.data.Datasets
datasets = mnist.as_dataset()
train_dataset, test_dataset = datasets['train'], datasets['test']

In [ ]:
# convert the Dataset to NumPy arrays and flatten the data
Xtrain, ytrain = [], []
for example in tfds.as_numpy(train_dataset):
  Xtrain.append(example['image'].flatten())
  ytrain.append(example['label'])

Xtrain, ytrain = np.asarray(Xtrain), np.asarray(ytrain)
Xtrain = Xtrain.astype(float)

# convert the test set to NumPy arrays and flatten the data
Xtest, ytest = [], []
for example in tfds.as_numpy(test_dataset):
  Xtest.append(example['image'].flatten())
  ytest.append(example['label'])

Xtest, ytest = np.asarray(Xtest), np.asarray(ytest)
Xtest = Xtest.astype(float)
X = np.vstack((Xtrain,Xtest))
y = np.hstack((ytrain,ytest))
X.shape, y.shape
# set random seed and shuffle the data
np.random.seed(1)
idx = np.arange(len(y))
np.random.shuffle(idx)
X, y = X[idx,:], y[idx]  

X.shape, y.shape 

((70000, 784), (70000,))

In [ ]:
# check if a column is all 0
id = [np.sum(Xtrain[:,i] != 0)>10 for i in range(28**2)]
# number of columns that mostly zero
print(28**2-np.sum(id))
# number of columns that has at least more than 10 non-zero
np.sum(id)
X = X[:, id]

135


In [ ]:
non_normal_rate(X)

In [ ]:
G = 10
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

## SOFT IMPUTE

In [ ]:
G = 10
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])

In [ ]:
soft_err.round(3)

array([[5.06105395e-05, 1.73117715e+03],
       [9.29168593e-05, 2.17783741e+03],
       [1.41229975e-04, 2.87814093e+03],
       [1.95521524e-04, 3.45418104e+03],
       [2.53988216e-04, 4.21292992e+03]])